# API Exploration Notebook

This notebook demonstrates how to work with two public APIs:

1. **SuperHero API**[](https://superheroapi.com/) — superhero character data (powerstats, biography, etc.)
2. **YH Finance API** (via https://financeapi.net/) — real-time & historical stock, ETF, crypto data

Goals:
- Understand API authentication (tokens / headers)
- Fetch data using requests
- Parse JSON responses
- Perform ETL (Extract → Transform → Load into pandas DataFrames)
- Basic analysis and visualization

**Prerequisites**
- SuperHero API: Sign up with GitHub at https://superheroapi.com/ to get an access token
- YH Finance API: Register at https://financeapi.net/ to get an API key (free tier available)

## 1. Imports & Setup

In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from datetime import datetime
import time  # for optional rate limiting

# ────────────────────────────────────────────────
# SUPERHERO API CONFIG
SUPERHERO_TOKEN = "YOUR_SUPERHERO_ACCESS_TOKEN_HERE"  # from https://superheroapi.com/
SUPERHERO_BASE = f"https://superheroapi.com/api/{SUPERHERO_TOKEN}"

# ────────────────────────────────────────────────
# YH FINANCE API CONFIG
FINANCE_API_KEY = "YOUR_YH_FINANCE_API_KEY_HERE"      # from https://financeapi.net/dashboard
FINANCE_BASE = "https://yfapi.net"
FINANCE_HEADERS = {"X-API-KEY": FINANCE_API_KEY}     # Key MUST be in headers

## Part A: SuperHero API

Fetch character profiles, powerstats, biography, and build comparisons.

### A.1 Test Connection (Batman – ID 70)

In [ ]:
test_url = f"{SUPERHERO_BASE}/70"
resp = requests.get(test_url)

if resp.status_code == 200:
    data = resp.json()
    print("SuperHero API working! Keys:", list(data.keys()))
    pprint(data, indent=2)
else:
    print("Error:", resp.status_code, resp.text)

### A.2 Helper: Get Character by ID

In [ ]:
def get_hero_by_id(hero_id):
    url = f"{SUPERHERO_BASE}/{hero_id}"
    resp = requests.get(url)
    if resp.status_code == 200:
        data = resp.json()
        if data.get('response') == 'success':
            return data
    print(f"Error fetching hero {hero_id}: {resp.status_code}")
    return None

### A.3 Collect Multiple Heroes into DataFrame (ETL)

In [ ]:
hero_ids = [70, 149, 332, 620]  # Batman, Captain America, Hulk, Spider-Man
heroes_data = []

for hid in hero_ids:
    hero = get_hero_by_id(hid)
    if hero:
        row = {
            'id': hero['id'],
            'name': hero['name'],
            'full_name': hero['biography'].get('full-name', 'N/A'),
            'publisher': hero['biography'].get('publisher', 'N/A'),
            'alignment': hero['biography'].get('alignment', 'N/A'),
            'intelligence': int(hero['powerstats'].get('intelligence', 0)),
            'strength':     int(hero['powerstats'].get('strength', 0)),
            'speed':        int(hero['powerstats'].get('speed', 0)),
            'durability':   int(hero['powerstats'].get('durability', 0)),
            'power':        int(hero['powerstats'].get('power', 0)),
            'combat':       int(hero['powerstats'].get('combat', 0)),
        }
        heroes_data.append(row)
    time.sleep(0.5)  # polite delay

df_heroes = pd.DataFrame(heroes_data)
df_heroes

### A.4 Visualization: Powerstats Comparison

In [ ]:
power_cols = ['intelligence', 'strength', 'speed', 'durability', 'power', 'combat']

df_heroes.plot(x='name', y=power_cols, kind='bar', figsize=(10,6))
plt.title('Superhero Powerstats Comparison')
plt.ylabel('Score (0–100)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Part B: YH Finance API (Stock Market Data)

Fetch quotes, historical prices, fundamentals — requires API key in headers.

### B.1 Test Connection (AAPL quote)

In [ ]:
test_url = f"{FINANCE_BASE}/v6/finance/quote"
params = {"symbols": "AAPL"}

resp = requests.get(test_url, headers=FINANCE_HEADERS, params=params)

if resp.status_code == 200:
    data = resp.json()
    print("Finance API working! Keys:", list(data.keys()))
    pprint(data, indent=2)
else:
    print("Error:", resp.status_code, resp.text)

### B.2 Helper: Get Real-time Quotes

In [ ]:
def get_stock_quotes(symbols_str):
    url = f"{FINANCE_BASE}/v6/finance/quote"
    params = {"symbols": symbols_str}
    resp = requests.get(url, headers=FINANCE_HEADERS, params=params)
    if resp.status_code == 200:
        return resp.json().get('quoteResponse', {}).get('result', [])
    print(f"Quote error: {resp.status_code}")
    return []

### B.3 Helper: Get Historical Data

In [ ]:
def get_historical(symbol, range_="1mo", interval="1d"):
    url = f"{FINANCE_BASE}/v8/finance/chart/{symbol}"
    params = {"range": range_, "interval": interval}
    resp = requests.get(url, headers=FINANCE_HEADERS, params=params)
    if resp.status_code == 200:
        result = resp.json().get('chart', {}).get('result', [{}])[0]
        ts = result.get('timestamp', [])
        quote = result.get('indicators', {}).get('quote', [{}])[0]
        df = pd.DataFrame({
            'date': [datetime.fromtimestamp(t) for t in ts],
            'open': quote.get('open', []),
            'high': quote.get('high', []),
            'low': quote.get('low', []),
            'close': quote.get('close', []),
            'volume': quote.get('volume', [])
        })
        return df
    print(f"History error for {symbol}: {resp.status_code}")
    return pd.DataFrame()

### B.4 ETL: Multiple Stocks into DataFrame

In [ ]:
stocks = "AAPL,MSFT,GOOGL,NVDA"
quotes = get_stock_quotes(stocks)

stock_rows = []
for q in quotes:
    row = {
        'symbol': q.get('symbol'),
        'name': q.get('longName', 'N/A'),
        'price': q.get('regularMarketPrice'),
        'change_pct': q.get('regularMarketChangePercent'),
        'volume': q.get('regularMarketVolume'),
        'market_cap': q.get('marketCap')
    }
    stock_rows.append(row)

df_stocks = pd.DataFrame(stock_rows)
df_stocks['market_cap_billions'] = (df_stocks['market_cap'] / 1e9).round(2)
df_stocks = df_stocks.sort_values('market_cap_billions', ascending=False)
df_stocks

### B.5 Visualization: Stock Prices & Historical Trend

In [ ]:
# Bar chart – current prices
df_stocks.plot(x='symbol', y='price', kind='bar', figsize=(8,5), color='teal')
plt.title('Current Stock Prices')
plt.ylabel('Price (USD)')
plt.show()

# Historical example – AAPL last month
df_aapl_hist = get_historical("AAPL")
if not df_aapl_hist.empty:
    df_aapl_hist.set_index('date')['close'].plot(figsize=(10,6), title='AAPL – 1 Month Close Price')
    plt.ylabel('Close Price (USD)')
    plt.show()

## Conclusion

We’ve now worked with two very different APIs:

- **SuperHero API** — token in URL path, JSON → structured DataFrame → powerstats viz
- **YH Finance API** — API key in headers, real-time + time-series data → ETL pipeline

Next steps could include:
- Error handling & retries
- Merging superhero “strength” with fictional stock tickers 😄
- Exporting DataFrames to CSV
- More endpoints (options chains, quote summary modules, superhero search)

Don’t forget to replace both tokens/keys before running!